## GA-output

Nb to test the output files and params of  the GASS

In [1]:
using PyCall
using DataFrames

#rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random
import JLD

np= pyimport("numpy")


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

┌ Info: Recompiling stale cache file /home/stephane/.julia/compiled/v1.1/GASS.ji for GASS [top-level]
└ @ Base loading.jl:1184


In [2]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    res= gass_optimization(cfg)
    
    return(res)
end

res= @main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 5 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##

##
## Creating the population...
1 1 -0.040676533237916535
Any[synthbeam(2.93973, 2.46878, 2.69398, 1.19076, 6.23534), 16.7828]
1 2 -2.519656374189799
Any[synthbeam(2.8428, 

10 1 2.7542527380731716
10 2 1.2919397416662548
10 3 -11.634293536455518
10 4 -0.1334209053389281

## Iteration: 2 
(8, 7)
[14, 1, 17, 23, 39, 29, 43, 22, 26, 13, 32, 5, 36, 21, 7, 31, 47, 24, 2, 19, 9, 37, 18, 20, 49, 27, 34, 11, 10, 42, 12, 44, 8, 41, 3, 45, 50, 46, 40, 48, 6, 4, 25, 28, 33, 38, 35, 15, 16, 30]
(7, 1)
[37, 15, 16, 5, 13, 30, 31, 24, 2, 19, 18, 20, 49, 10, 43, 14, 8, 41, 46, 1, 17, 6, 23, 39, 29, 22, 3, 4, 27, 21, 36, 7, 32, 25, 12, 28, 34, 33, 48, 26, 9, 47, 11, 38, 50, 42, 35, 45, 40, 44]
(1, 7)
[24, 18, 43, 14, 8, 41, 46, 1, 17, 5, 29, 13, 2, 19, 49, 30, 21, 36, 23, 7, 32, 20, 31, 6, 39, 10, 3, 4, 27, 25, 12, 28, 34, 33, 48, 22, 26, 9, 47, 11, 38, 50, 42, 35, 45, 40, 44, 15, 16, 37]
(1, 7)
[36, 32, 5, 13, 21, 7, 31, 24, 2, 19, 18, 20, 49, 10, 43, 14, 8, 46, 1, 17, 6, 23, 39, 29, 22, 3, 4, 27, 25, 12, 28, 34, 33, 48, 26, 9, 47, 11, 38, 50, 42, 15, 35, 45, 40, 44, 41, 16, 30, 37]
(7, 1)
[37, 15, 5, 13, 21, 7, 31, 24, 2, 19, 18, 20, 49, 10, 43, 14, 8, 41, 46, 1, 17, 6

1 1 1.9002519007211134
1 2 -1.8603969473813016
1 3 -22.090795806969908
1 4 -18.014269272942297
2 1 -4.745065853700494
2 2 -1.2898373598607331
2 3 -12.701895515269246
2 4 -17.767708227571973
3 1 2.8620414249971233
3 2 0.6920683637016753
3 3 -21.703833763679206
3 4 -12.559711975349114
4 1 2.5336019378163104
4 2 -1.2281992744250874
4 3 -14.10186469918115
4 4 -17.767708227571973
5 1 -0.333735389561054
5 2 -4.043442033697698
5 3 -13.622631276298897
5 4 -12.277251862582627
fitness of crossover..
[43, 6, 1, 17, 23, 39, 29, 22, 2, 36, 26, 5, 13, 21, 7, 31, 47, 24, 10, 19, 3, 37, 18, 20, 49, 27, 34, 11, 32, 35, 14, 8, 41, 9, 46, 4, 25, 28, 33, 42, 38, 50, 48, 15, 12, 45, 40, 44, 16, 30]
Array{Int64,1}[[43, 6, 1, 17, 23, 39, 29, 22, 2, 36, 26, 5, 13, 21, 7, 31, 47, 24, 10, 19, 3, 37, 18, 20, 49, 27, 34, 11, 32, 35], [14, 8, 41, 9, 46, 4, 25, 28, 33, 42], [38, 50, 48, 15, 12, 45, 40], [44, 16, 30]]
6 1 1.6380248070794887
6 2 -2.2591681978944393
6 3 -12.050506337337225
6 4 -3.895584916487944
[44, 

5-element Array{Any,1}:
 population(1, Array{Int64,1}[[17, 1, 37, 15, 16, 14, 43, 22, 24, 5  …  32, 20, 31, 6, 39, 10, 3, 4, 27, 25] [12, 28, 34, 33, 48, 26, 9, 47, 11, 38] [50, 42, 35, 45, 40, 44, 41] [46, 8, 18]; [26, 36, 32, 5, 13, 21, 7, 31, 47, 24  …  10, 42, 43, 12, 44, 14, 8, 41, 3, 45] [50, 46, 40, 48, 1, 17, 6, 23, 35, 39] [29, 22, 38, 15, 33, 28, 4] [16, 30, 37]; … ; [36, 32, 5, 13, 21, 7, 19, 24, 2, 31  …  50, 46, 40, 1, 17, 10, 3, 4, 27, 25] [12, 28, 34, 33, 48, 26, 9, 47, 11, 38] [6, 23, 35, 39, 29, 22, 15] [16, 30, 37]; [14, 16, 22, 42, 8, 40, 24, 43, 9, 15  …  32, 20, 31, 6, 39, 10, 3, 4, 27, 25] [12, 28, 34, 33, 48, 26, 5, 47, 11, 38] [50, 1, 44, 45, 13, 17, 41] [29, 18, 37]], [1.71316 0.846497 -14.9751 -12.7215; 1.86517 1.16199 -12.5903 -12.2773; … ; -0.559964 0.846497 -12.0132 -12.2773; 2.75425 1.29194 -11.6343 -0.133421], [6.28423, 5.4601, 4.6399, 4.45085, 3.22964, -0.0921614, 6.67003, 5.5771, 6.00099, 1.93038], Dict{String,Float64}[Dict("mrs"=>9.86067,"ar"=>2.95307,

In [3]:
println(res[1].age)
println(res[1].fitness)
println(res[5].score)

JLD.save("test.jld", "evol", res)

1
[1.71316 0.846497 -14.9751 -12.7215; 1.86517 1.16199 -12.5903 -12.2773; 2.65418 1.24515 -12.7649 -9.69399; 2.39256 -3.23792 -17.6418 0.683782; 2.77317 -2.63441 -12.2891 -0.768154; 3.04383 1.24515 -11.8633 7.94298; -0.47927 0.846497 -14.7701 -12.2773; -5.22272 1.57555 -12.8977 -5.76356; -0.559964 0.846497 -12.0132 -12.2773; 2.75425 1.29194 -11.6343 -0.133421]
[10.0163, 9.98044, 9.12613, 7.67736, 7.64104, 13.2474, 11.5697, 7.02929, 4.70003, 8.65599]


In [4]:
a = JLD.load("test.jld")
a["evol"][5].subarr[5,2]

10-element Array{Int64,1}:
 12
 14
  8
 41
  3
 45
 50
 46
 40
 48

In [9]:
### save array in CASA format..
###
function _save_CASAarr(fileprefix::String, cfg::cfg, p::population)
    println(cfg.arr)
end

_save_CASAarr (generic function with 1 method)

In [12]:
inpfile= "../master/data/GA_Inputs_O-3.txt.julia"
a= read_cfg(inpfile , verbose=true)
println(typeof(res))
_save_CASAarr("toto", a, res)

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 5 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
Array{Any,1}


MethodError: MethodError: no method matching _save_CASAarr(::String, ::cfg, ::Array{Any,1})
Closest candidates are:
  _save_CASAarr(::String, ::cfg, !Matched::population) at In[9]:4